This notebook contains the visual representation of creating the other tables and expanding some existing tables.  Uses FiveThirtyEight's common names dataset to randomly generate names.

Datasets: 

(1) https://www.kaggle.com/zynicide/nfl-football-player-stats  
(2) https://www.kaggle.com/fivethirtyeight/fivethirtyeight-most-common-name-dataset

In [1]:
import pickle
import pandas as pd
import numpy as np

from random import sample
from random import shuffle

## Re-open Old Pickles

In [2]:
with open(r'../Data/Pickles/offense.pickle', 'rb') as f:
    offense = pickle.load(f)

In [3]:
with open(r'../Data/Pickles/defense.pickle', 'rb') as f:
    defense = pickle.load(f)

In [4]:
offense.head()

,gameNumber,passYards,rushYards,recYards,interceptions,touchdowns,playerID,position
4,1,0,0,49,0,0,Emmanuel Sanders,WR
5,2,0,0,39,0,0,Emmanuel Sanders,WR
6,3,0,0,117,0,2,Emmanuel Sanders,WR
7,4,0,0,88,0,1,Emmanuel Sanders,WR
8,5,0,0,80,0,0,Emmanuel Sanders,WR


In [5]:
defense.head()

,playerID,gameNumber,sacks,interceptions,touchdowns,safeties,blocks
0,Arizona Cardinals,1,2.0,0,0,0,0
1,Arizona Cardinals,2,2.0,2,0,0,0
2,Arizona Cardinals,3,2.0,1,0,1,0
3,Arizona Cardinals,4,2.0,0,0,0,0
4,Arizona Cardinals,5,2.0,0,0,0,0


## Create users

In [6]:
names = pd.read_csv(r'../Data/Datasets/names.csv')

In [7]:
names = names[['FirstName', 'Surname']]
fnames = list(np.unique(names['FirstName'].to_numpy()))
lnames = list(np.unique(names['Surname'].to_numpy()))

In [8]:
print(f'{len(fnames)} first names')
print(f'{len(lnames)} last names')

20 first names
20 last names


To generate 20 names, let's sample 4 last names and 5 first names

In [9]:
sample_fnames = sample(fnames, 5)
sample_lnames = sample(lnames, 4)

list(zip(sample_fnames, sample_lnames))

[('Maria', 'Jones'),
 ('Michael', 'Wilson'),
 ('Joseph', 'Anderson'),
 ('Richard', 'Lee')]

In [10]:
generated_names = []
for f in sample_fnames:
    for l in sample_lnames:
        generated_names.append(f + ' ' + l)

In [11]:
print(generated_names)

['Maria Jones', 'Maria Wilson', 'Maria Anderson', 'Maria Lee', 'Michael Jones', 'Michael Wilson', 'Michael Anderson', 'Michael Lee', 'Joseph Jones', 'Joseph Wilson', 'Joseph Anderson', 'Joseph Lee', 'Richard Jones', 'Richard Wilson', 'Richard Anderson', 'Richard Lee', 'James Jones', 'James Wilson', 'James Anderson', 'James Lee']


In [12]:
shuffle(generated_names)
print(generated_names)

['Richard Wilson', 'Maria Anderson', 'Richard Lee', 'Maria Jones', 'Michael Lee', 'James Jones', 'Joseph Jones', 'Richard Anderson', 'James Wilson', 'Richard Jones', 'Joseph Anderson', 'Michael Jones', 'James Anderson', 'Joseph Lee', 'Maria Wilson', 'Joseph Wilson', 'Michael Wilson', 'Maria Lee', 'Michael Anderson', 'James Lee']


## Make "league" Table

In [13]:
league1 = generated_names[:10]
league2 = generated_names[10:]

In [14]:
league = pd.DataFrame([[1, 'family'], [2, 'competitive']], columns=['leagueID', 'leagueType'])
league.set_index('leagueID')

,leagueType
leagueID,
1,family
2,competitive


In [15]:
league.to_csv(r'../Data/CSV/leagues.csv', header=False, index=False)

## Sample players to help form contracts

In [16]:
l1_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league1))
l1_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league1))
l1_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league1))
l1_defense = sample(list(defense['playerID'].unique()), len(league1))

In [17]:
l2_qbs = sample(list(offense[offense['position'] == 'QB']['playerID'].unique()), len(league2))
l2_rbs = sample(list(offense[offense['position'] == 'RB']['playerID'].unique()), 2*len(league2))
l2_wrs = sample(list(offense[offense['position'] == 'WR']['playerID'].unique()), 2*len(league2))
l2_defense = sample(list(defense['playerID'].unique()), len(league2))

## Make "contract" table

In [18]:
contract = pd.DataFrame(l1_qbs+l1_rbs+l1_wrs+l1_defense+l2_qbs+l2_rbs+l2_wrs+l2_defense, columns=['playerID'])
contract['owner'] = league1 + 2*league1 + 2*league1 + league1 + league2 + 2*league2 + 2*league2 + league2
contract['startWeek'] = 1
contract['endWeek'] = 16
contract['isActive'] = 1

In [19]:
def name_to_id(name):
    return name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3]

In [20]:
contract['owner'] = contract['owner'].apply(name_to_id)

In [21]:
contract

,playerID,owner,startWeek,endWeek,isActive
0,Matthew Stafford,wilsonric,1,16,1
1,Blake Bortles,andersonmar,1,16,1
2,Brock Osweiler,leeric,1,16,1
3,Terrelle Pryor,jonesmar,1,16,1
4,Mark Sanchez,leemic,1,16,1
...,...,...,...,...,...
115,Baltimore Ravens,wilsonjos,1,16,1
116,Indianapolis Colts,wilsonmic,1,16,1
117,Los Angeles Chargers,leemar,1,16,1
118,Washington Redskins,andersonmic,1,16,1


## Create "user" table

In [22]:
user = pd.DataFrame([(name.split(' ')[0], name.split(' ')[1], name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3]) 
                     for name in generated_names], 
                    columns=['nameFirst', 'nameLast', 'userID'])

In [23]:
user.head()

,nameFirst,nameLast,userID
0,Richard,Wilson,wilsonric
1,Maria,Anderson,andersonmar
2,Richard,Lee,leeric
3,Maria,Jones,jonesmar
4,Michael,Lee,leemic


In [24]:
user.to_csv(r'../Data/CSV/users.csv', index=False, header=False)

## Create "team" table

In [25]:
leagues1 = {name: 1 for name in league1}
leagues2 = {name: 2 for name in league2}

leagues1.update(leagues2)

In [26]:
teams = pd.DataFrame([(f'Team{number}', name.split(' ')[1].lower()+name.split(' ')[0].lower()[:3], leagues1[name]) 
                      for number, name in enumerate(generated_names)],
                     columns=['teamID', 'userID', 'leagueID'])

In [27]:
teams

,teamID,userID,leagueID
0,Team0,wilsonric,1
1,Team1,andersonmar,1
2,Team2,leeric,1
3,Team3,jonesmar,1
4,Team4,leemic,1
5,Team5,jonesjam,1
6,Team6,jonesjos,1
7,Team7,andersonric,1
8,Team8,wilsonjam,1
9,Team9,jonesric,1


In [28]:
teams.to_csv(r'../Data/CSV/team.csv', index=False, header=False)

## Merge teams and contracts and drop names

In [29]:
real_contracts = pd.merge(contract, teams.rename({'userID': 'owner'}, axis=1), on='owner').drop(['owner', 'leagueID'], axis=1)

In [30]:
real_contracts

,playerID,startWeek,endWeek,isActive,teamID
0,Matthew Stafford,1,16,1,Team0
1,Mike James,1,16,1,Team0
2,Damien Williams,1,16,1,Team0
3,Willie Snead,1,16,1,Team0
4,T.Y. Hilton,1,16,1,Team0
...,...,...,...,...,...
115,Kenneth Dixon,1,16,1,Team19
116,Devontae Booker,1,16,1,Team19
117,Eli Rogers,1,16,1,Team19
118,Russell Shepard,1,16,1,Team19


In [31]:
real_contracts.to_csv(r'../Data/CSV/contracts.csv', index=False, header=False)